## Chart the RSEI Score and Pounds post-treatment values for selected chemicals
Compare all USA facilities in RSEI with facilities offered a reporting exemption.

Chemicals selected (ChemicalNumber in parentheses):

'1,2,4-Trichlorobenzene', (564)

'1,2,4-Trimethylbenzene', (575)

'Benzene', (51)

'Hexachlorobenzene', (321)

'Mercury', (359)

'Mercury compounds', (360)

'Polychlorinated biphenyls ', (474)

'Polycyclic aromatic compounds' (609)

‘Ethylene oxide’, (293)

Group the first four benzene-related chemicals in the Benzene chart.

Combine Mercury and Mercury compounds



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

The file All-submissions-elements-4-chemicals.csv is produced by the chemicals-by-fac.py program.

The file Exemptions-facility-chemicals is produced by running the RSEI.ipynb notebook, with the Exemptions-FullList.csv facilities (FRSID List), Media=[1,2] (air releases).

In [ ]:
usa_filename = '/home/steve/workspace/EDGI/RSEI/CSVs/All-submissions-elements-4-chemicals.csv'
exemptions_filename = '/home/steve/workspace/EDGI/RSEI/CSVs/Exemptions-facility_chemicals.csv'

In [ ]:
usa = pd.read_csv(usa_filename)
# usa.set_index('Year', inplace=True)
usa = usa[usa['Media'].isin([1,2])]
usa['Who'] = 'USA'
# usa.rename(columns={'USA' : data_column}, inplace=True)

exemptions = pd.read_csv(exemptions_filename)
# usa.set_index('Year', inplace=True)
exemptions['Who'] = 'Exemptions'
# usa.rename(columns={'USA' : data_column}, inplace=True)

usa = usa[~usa['FacilityID'].isin(exemptions['FacilityID'])]

In [ ]:

def make_chart(graph_title, data_column, y_axis_label, df):
    #set seaborn plotting aesthetics
    sns.set(style='white')

    #create grouped bar chart

    fig, ax = plt.subplots(figsize=(10,10))
    try:
        g = sns.barplot(x='SubmissionYear', y=data_column, hue='Who', data=df)
        g.set_title(graph_title)
        ax.set_ylabel(y_axis_label)
        return g
    except TypeError as te:
        print("TypeError: {}".format(str(te)))
        return None

In [ ]:
chemicals = {51 : 'Benzene', 
             359 : 'Mercury', 
             474 : 'Polychorinated biphenyls', 
             609 : 'Polycyclic aromatic compounds', 
             293 : 'Ethylene oxide'}

for key, value in chemicals.items():
    if key == 51:
        chemical_usa = usa[usa['ChemicalNumber'].isin([51, 321, 564, 575])]
        chemical_exemptions = exemptions[exemptions['Chemical'].isin(['1,2,4-Trichlorobenzene', 
                                                                      '1,2,4-Trimethylbenzene', 
                                                                      'Hexachlorobenzene',
                                                                      'Benzene'])]
    elif key == 359:
        chemical_usa = usa[usa['ChemicalNumber'].isin([359, 360])]
        chemical_exemptions = exemptions[exemptions['Chemical'].isin(['Mercury', 
                                                                      'Mercury compounds'])]
    else:
        chemical_usa = usa[usa['ChemicalNumber'] == key]
        chemical_exemptions = exemptions[exemptions['Chemical'] == value]

    exemptions_count = len(chemical_exemptions['FacilityID'].unique())
    usa_count = len(chemical_usa['FacilityID'].unique())
    print(f'{value} - exemptions_count={exemptions_count}, usa_count={usa_count}')

    exemptions_to_chart = chemical_exemptions.groupby(['SubmissionYear'], as_index=False).agg({'Score' : 'sum', 'PoundsReleased' : 'sum', 'PoundsPT' : 'sum'})
    exemptions_to_chart['Score'] /= exemptions_count
    exemptions_to_chart['PoundsReleased'] /= exemptions_count
    exemptions_to_chart['PoundsPT'] /= exemptions_count
    usa_to_chart = chemical_usa.groupby(['SubmissionYear'], as_index=False).agg({'Score' : 'sum', 'PoundsReleased' : 'sum', 'PoundsPT' : 'sum'})
    usa_to_chart['Score'] /= usa_count
    usa_to_chart['PoundsReleased'] /= usa_count
    usa_to_chart['PoundsPT'] /= usa_count

    usa_to_chart['Who'] = 'USA'
    exemptions_to_chart['Who'] = 'Exemptions'

    to_chart = pd.concat([usa_to_chart, exemptions_to_chart], axis=0)
    to_chart['SubmissionYear'] = to_chart['SubmissionYear'].astype(int)

    data_column = 'Score'
    graph_title = f'{value} - Exemptions and All Other USA Facilities'
    y_axis = 'Average RSEI Score - per facility'
    display(make_chart(graph_title=graph_title, data_column=data_column, y_axis_label=y_axis, df=to_chart))
    
    data_column = 'PoundsPT'
    graph_title = f'{value} - Exemptions and All Other USA Facilities'
    y_axis = 'Average Pounds Post-treatment - per facility'
    display(make_chart(graph_title=graph_title, data_column=data_column, y_axis_label=y_axis, df=to_chart))